<a href="https://colab.research.google.com/github/poojamahajan0712/GenAI/blob/main/LLM/3_opensource_LLM_Mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## prerequisites -- hf account, access token and request for model access based on model.
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [3]:
# from huggingface_hub import login

# login(key)

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

Device set to use cuda:0


In [7]:
prompt = "Can you explain the difference between equity and equality"

sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=500,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Can you explain the difference between equity and equality in terms of social justice? Equity refers to the idea that individuals or groups should receive resources or opportunities in accordance with their specific needs, rather than a one-size-fits-all approach. Equality, on the other hand, is the idea that all individuals or groups should be treated identically, regardless of their specific needs or circumstances. In terms of social justice, equity is often seen as a more nuanced and complex approach that acknowledges the systemic inequalities and discrimination that exist in society and seeks to address them through targeted policies and resources. Equality, while well-intentioned, can overlook the unique challenges and barriers faced by marginalized groups and may not effectively address the root causes of inequality. Ultimately, both equity and equality are important concepts in the pursuit of social justice, and a balanced approach that incorporates both principles is often seen

In [ ]:
# https://www.datacamp.com/tutorial/mistral-7b-tutorial